In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [6]:
import numpy as np
from sklearn.cluster import KMeans

def stratified_cluster_split(X, y, test_size=0.5, random_state=42):
    X_sub_list = []
    y_sub_list = []
    X_rest_list = []
    y_rest_list = []

    classes = np.unique(y)

    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        X_c = X[y == c]
        y_c = y[y == c]

        kmeans = KMeans(n_clusters=2, random_state=random_state)
        cluster_labels = kmeans.fit_predict(X_c)

        cluster_0_idx = np.where(cluster_labels == 0)[0]
        cluster_1_idx = np.where(cluster_labels == 1)[0]

        if len(cluster_0_idx) <= len(cluster_1_idx):
            base_idx = cluster_0_idx
        else:
            base_idx = cluster_1_idx

        n_select = int(len(X_c) * test_size)
        if n_select == 0:
            continue

        if len(base_idx) >= n_select:
            selected_idx = rng.choice(base_idx, size=n_select, replace=False)
        else:
            extra_needed = n_select - len(base_idx)
            other_idx = np.setdiff1d(np.arange(len(X_c)), base_idx)
            extra_idx = rng.choice(other_idx, size=extra_needed, replace=False)
            selected_idx = np.concatenate([base_idx, extra_idx])

        rest_idx = np.setdiff1d(np.arange(len(X_c)), selected_idx)

        X_sub_list.append(X_c[selected_idx])
        y_sub_list.append(y_c[selected_idx])
        X_rest_list.append(X_c[rest_idx])
        y_rest_list.append(y_c[rest_idx])

    X_sub = np.concatenate(X_sub_list, axis=0)
    y_sub = np.concatenate(y_sub_list, axis=0)
    X_rest = np.concatenate(X_rest_list, axis=0)
    y_rest = np.concatenate(y_rest_list, axis=0)

    return X_rest, X_sub, y_rest, y_sub

_, X_50, _, y_50 = stratified_cluster_split(x_test_adv_0_1, y_label_0_1, test_size=0.5, random_state=42)


In [7]:
X_50.shape

(512867, 56)

In [8]:
from xgboost import XGBClassifier

print('xgb')
xgb50 = XGBClassifier()
xgb50.fit(X_50, y_50)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf50 = RandomForestClassifier()
rf50.fit(X_50, y_50)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt50 = DecisionTreeClassifier()
dt50.fit(X_50, y_50)

xgb
RF
DT


DecisionTreeClassifier()

In [9]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Cluster/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Cluster/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [36570   151  2204   260   104  2445     1   116 71845   216    59]
(36570, 56) (36570,)
Save 0 to baseline_Def1.npy
(151, 56) (151,)
Save 1 to baseline_Def2.npy
(2204, 56) (2204,)
Save 2 to baseline_Def3.npy
(260, 56) (260,)
Save 3 to baseline_Def4.npy
(104, 56) (104,)
Save 4 to baseline_Def5.npy
(2445, 56) (2445,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(116, 56) (116,)
Save 7 to baseline_Def8.npy
(71845, 56) (71845,)
Save 8 to baseline_Def9.npy
(216, 56) (216,)
Save 9 to baseline_Def10.npy
(59, 56) (59,)
Save 10 to baseline_Def11.npy
Execution Time: 0.288177 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [193988    232   1612    892    850 110742     24   1722  31811     38
      2]
(193988, 56) (193988,)
Save 0 to BIM_Def1.npy
(232, 56) (232,)
Save 1 to BIM_Def2.npy
(1612, 56) (1612,)
Save 2 to BIM_Def3.npy
(892, 56) (892,)
Save 3 to BIM_Def4.npy
(850, 56) (850,)
Save 4 to BIM_Def5.npy
(110742, 56) (110

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [36728   170  2116   485   182  2583   247 71197   193    70]
(36728, 56) (36728,)
Save 0 to baseline_Def1.npy
(170, 56) (170,)
Save 1 to baseline_Def2.npy
(2116, 56) (2116,)
Save 2 to baseline_Def3.npy
(485, 56) (485,)
Save 3 to baseline_Def4.npy
(182, 56) (182,)
Save 4 to baseline_Def5.npy
(2583, 56) (2583,)
Save 5 to baseline_Def6.npy
(247, 56) (247,)
Save 7 to baseline_Def8.npy
(71197, 56) (71197,)
Save 8 to baseline_Def9.npy
(193, 56) (193,)
Save 9 to baseline_Def10.npy
(70, 56) (70,)
Save 10 to baseline_Def11.npy
Execution Time: 2.393763 seconds
(341913,)
[0 1 2 3 4 5 7 8] [168472    144   2246    735     15 142916     25  27360]
(168472, 56) (168472,)
Save 0 to BIM_Def1.npy
(144, 56) (144,)
Save 1 to BIM_Def2.npy
(2246, 56) (2246,)
Save 2 to BIM_Def3.npy
(735, 56) (735,)
Save 3 to BIM_Def4.npy
(15, 56) (15,)
Save 4 to BIM_Def5.npy
(142916, 56) (142916,)
Save 5 to BIM_Def6.npy
(25, 56) (25,)
Save 7 to BIM_Def8.npy
(27360, 56) (27360,)
Sav

In [11]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt50.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Cluster/UNSW_Input50/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Cluster/UNSW_Input50/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [48563   262  2208   730   357  2866   296 58360   264    65]
(48563, 56) (48563,)
Save 0 to baseline_Def1.npy
(262, 56) (262,)
Save 1 to baseline_Def2.npy
(2208, 56) (2208,)
Save 2 to baseline_Def3.npy
(730, 56) (730,)
Save 3 to baseline_Def4.npy
(357, 56) (357,)
Save 4 to baseline_Def5.npy
(2866, 56) (2866,)
Save 5 to baseline_Def6.npy
(296, 56) (296,)
Save 7 to baseline_Def8.npy
(58360, 56) (58360,)
Save 8 to baseline_Def9.npy
(264, 56) (264,)
Save 9 to baseline_Def10.npy
(65, 56) (65,)
Save 10 to baseline_Def11.npy
Execution Time: 0.085297 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [207062   2816  21063   3184   1342  68352    375  10606  23583    541
   2989]
(207062, 56) (207062,)
Save 0 to BIM_Def1.npy
(2816, 56) (2816,)
Save 1 to BIM_Def2.npy
(21063, 56) (21063,)
Save 2 to BIM_Def3.npy
(3184, 56) (3184,)
Save 3 to BIM_Def4.npy
(1342, 56) (1342,)
Save 4 to BIM_Def5.npy
(68352, 56) (68352,)
Save 5 to BIM_Def6.npy
(375, 56) (375,

In [12]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}50")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_Cluster/UNSW_Input50/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [13]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input50_Cluster.csv")

In [14]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB50,36570,151,2204,260,104,2445,1,116,71845,216,59,113971
1,XGB50,193988,232,1612,892,850,110742,24,1722,31811,38,2,341913
2,XGB50,153910,165,1536,874,308,153501,21,1447,30123,14,14,341913
3,XGB50,193988,232,1612,892,850,110742,24,1722,31811,38,2,341913
4,XGB50,88935,505,2254,477,187,229308,452,17709,1699,340,47,341913
5,XGB50,186760,418,4259,1456,834,117860,25,1518,28432,249,102,341913
6,XGB50,105262,468,6545,766,312,7361,4,345,220065,615,170,341913
7,XGB50,287465,43,256,505,105,9957,1,74,41276,1353,878,341913
8,XGB50,178533,237,1029,1166,714,126276,60,1464,32398,31,5,341913
9,XGB50,170105,145,1634,1558,966,132777,21,544,34121,33,9,341913
